In [ ]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from data_preprocessing import *
%load_ext autoreload
%autoreload 2

# Support Vector Machines
## Classification Using SVM
Load dataset. We will re-use the CERN dataset from project 1, available from https://inclass.kaggle.com/c/epfml-project-1/data

In [ ]:
from proj1_helpers import load_csv_data
DATA_TRAIN_PATH = '../data/train.csv' # TODO: download train data and supply path here 
y, x, ids = load_csv_data(DATA_TRAIN_PATH)
# TODO: convert labels to -1,1 ?
print(x.shape)

## Note: This is the raw dataset, you can also work with your modified features if you prefer

In [ ]:
x = data_fix_and_standardize(x)
print(x.shape)
y=y[1:100]
x=x[1:100,:]
print(x.shape)

In [ ]:
def calculate_cost(y, x, w, lambda_):
    """compute the full cost (the primal objective), that is loss plus regularizer."""
    # Here x is the full dataset matrix, and y are the corresponding +1 or -1 labels
    # ***************************************************
    loss=0
    for i in range(len(y)):
        v = y[i]*np.dot(x[i,:].T,w)
        loss += max(0,1-v)
        
    loss=loss+lambda_/2 * pow(np.linalg.norm(w),2)
    return loss
    
    # ***************************************************


In [ ]:
w = (np.linalg.inv(np.transpose(x).dot(x)).dot(np.transpose(x))).dot(y)
lambda_=0
error=calculate_cost(y,x,w, lambda_)
print(error)

## Stochastic Gradient Descent for SVM

Compute the (stochastic) subgradient for the n-th summand of the SVM optimization objective

In [ ]:
def calculate_gradient(y, x, w, lambda_, n): # Juste calcul du gradient
    """compute the stochastic gradient of loss plus regularizer."""
    # Here x is one datapoint, and y is the corresponding +1 or -1 label
    """prod=np.dot(x.T,w)
    e=y-prod
    stoch_grad=(-e/N)*x"""
   
    grad=np.zeros((len(w),1))
    for i in range(len(grad)):
        if y*np.dot(x.T,w)<1:     #car ici x n est que en position i et y le sol associée
            grad[i]=-x[i]*y+lambda_*w[i]
        else:
            grad[i]=0
        
        
    return grad
    # ***************************************************

    # Be careful about the constant N(size) term! The complete objective for SVM is a sum, not an average as in earlier SGD examples!
   

Implement stochastic gradient descent: Pick a data point uniformly at random and update w based on the gradient for the n-th summand of the objective

In [ ]:
def sgd_for_svm_demo(y, x):
    # ***************************************************
    # INSERT YOUR CODE HERE
    # classify the data by SGD for SVM: TODO
    # ***************************************************
    max_iter = 10000
    gamma = 0.01
    lambda_ = 1.0 / y.shape[0]  # or set to a different value, try cross-validation!
    N=len(y)
    w = np.zeros((x.shape[1], 1))
    print(x.shape)
    for iter in range(max_iter):
        n = np.random.randint(N) 
        loss =  calculate_cost(y,x,w,lambda_)
        grad =  calculate_gradient(y[n],x[n,:],w,lambda_,n)#/np.linalg.norm(grad)
        w = w -gamma*grad
        
        if iter % 1000 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
    
    print("Objective = {l}".format(l=calculate_cost(y, x, w, lambda_)))

sgd_for_svm_demo(y, x)

## Coordinate Descent (Ascent) for SVM

Compute the closed-form update for the n-th variable alpha, in the dual optimization problem, given alpha and the current corresponding w

In [ ]:
def calculate_coordinate_update(y, x, lambda_, alpha, w, n): # alpha [0,1]
    # Here x is one datapoint, and y is the corresponding +1 or -1 label
    # 
    # ***************************************************
    alphac=1*alpha
    xn=np.reshape(x[n,:],(1,x.shape[1]))
    old=alpha[n]*(1-1/lambda_*y[n]*np.dot(xn,(np.dot(x.T,np.dot(np.diag(y),alpha)))))+lambda_/2 *pow(np.linalg.norm(1/lambda_*np.dot(x.T,np.dot(np.diag(y),alpha))),2)
    for u in range(0,11):
        u=u/10
        alphac[n]=u

        new=alphac[n]*(1-1/lambda_*y[n]*np.dot(xn,(np.dot(x.T,np.dot(np.diag(y),alphac)))))+lambda_/2 *pow(np.linalg.norm(1/lambda_*np.dot(x.T,np.dot(np.diag(y),alphac))),2)
        if new<old:
            alpha[n]=u
        old=new
    # ***************************************************
    return alpha

In [ ]:
def coordinate_descent_for_svm_demo(y, x):
    # ***************************************************
    # INSERT YOUR CODE HERE
    # classify the data by SGD for SVM: TODO
    # ***************************************************
    max_iter = 10000
    gamma = 0.00001
    lambda_ = 1.0 / y.shape[0]
    N=len(y)

    w = np.zeros((x.shape[1], 1))
    alpha=np.zeros((N,1))
    for iter in range(max_iter):
        n = np.random.randint(N) 
        loss =  calculate_cost(y,x,w,lambda_)
        print(sum(w))
        alpha = calculate_coordinate_update(y,x,lambda_,alpha,w,n)
        w = 1/lambda_*np.dot(x.T,np.dot(np.diag(y),alpha))
        print(sum(w))
        if iter % 100 == 0:
            print("Current iteration={i}, the loss={l}".format(i=iter, l=loss))
    
    print("Primal objective = {l}".format(l=calculate_cost(y, x, w, lambda_)))

coordinate_descent_for_svm_demo(y, x)